### Week 3 Assignment Part 1

Build a code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

#### Reference: 

https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas

Firstly, import the library to connect to the Wikipedia page and fetch the contents of that page

In [1]:
# import the library we use to open URLs
import urllib.request

Next, specify the URL of the Wikipedia page that we are looking to scrape

Using the urllib.request library, query the page and put the HTML data into a variable

In [2]:
# specify which URL/web page to be scrape
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

Next we want to import the functions from Beautiful Soup which will let us parse and work with the HTML we fetched from our Wiki page

In [3]:
# import the BeautifulSoup library to parse HTML and XML documents
from bs4 import BeautifulSoup

Then we use Beautiful Soup to parse the HTML data we stored in our ‘url’ variable and store it in a new variable called ‘soup’ in the Beautiful Soup format. Jupyter Notebook prefers we specify a parser format so we use the “lxml” library option.

In [4]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

To get an idea of the structure of the underlying HTML in our web page, we can view the code in two ways: 

a) right click on the web page itself and click View Source or 

b) use Beautiful Soup’s prettify function and check it out right there in our Jupyter Notebook.

In [5]:
# soup.prettify() # --> This is a very long list

Starting with an HTML `<table>` tag with a class identifier of `wikitable sortable`. 

The rows start and end with `<tr>` and `</tr>` tags respectively.

The top row of headers has `<th>` tags while the data rows beneath for each club has `<td>` tags. 

It’s in these tags that we will tell Python to extract our data from.

Knowing that the data resides within an HTML table, we need to send Beautiful Soup off to retrieve all instances of the `<table>` tag within the page and add them to an array called `all_tables`.

In [6]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
# all_tables # --> THis is a very long list

Looking through the output of `all_tables` we can again see that the class id of our chosen table is `wikitable sortable`

We can use this to get Beautiful Soup to only bring back the table data for this particular table and keep that in a variable called `right_table`

In [7]:
right_table=soup.find('table', class_='wikitable sortable')

# right_table # --> In this case, right_table is similar to all_tables

### Start looping through the rows to get the required data

There are 3 columns in the table that we want to scrape the data from. 

Hence we will set up 3 empty lists (A, B and C) to store our data in.

Use the Beautiful Soup ‘find_all’ function again and set it to look for the string ‘tr’. 

We will then set up a `for` loop for each row within that array and set Python to loop through the rows, one by one.

Within the loop we are going to use find_all again to search each row for <td> tags with the ‘td’ string. 
    
We will add all of these to a variable called ‘cells’ and then check to make sure that there are 3 items in our ‘cells’ array (i.e. one for each column).

If there are then we use the find(text=True)) option to extract the content string from within each `<td>` element in that row and add them to the A-C lists we created at the start of this step.

In [8]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

### Introducing pandas and dataframes

Import pandas and create a dataframe with it, assigning each of the lists A-C into a column with the name of our source table columns i.e. PostalCode, Borough, and Neighborhood.

In [9]:
import pandas as pd

In [10]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


#### Remove all the `\n` in the dataframe

In [11]:
# Remove all the \n in the dataframe
df = df.replace(r'\n',  ' ', regex=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Remove the columns that contains `Not assigned`

In [12]:
df = df[~df['Borough'].str.contains("Not assigned")]
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Reset the index

In [13]:
df.reset_index(inplace=True)
df

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Remove the column named `index`

In [14]:
df = df.drop(columns = "index")

# Visualize the first 12 rows of dataframe
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Save the dataframe for Part 2

In [15]:
df.to_csv("Toronto_Post.csv")

#### This is the last cell of this notebook 

Use the `.shape` method to print the number of rows of the dataframe

In [16]:
df.shape

(103, 3)